In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
# define a function to compute sentiments of the received tweets
from pyspark.ml import PipelineModel
pipelineFit=PipelineModel.load("preprocessingAndLR")
pipelineFitBBC=PipelineModel.load("preprocessingAndCategory")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/02 21:33:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/02 21:33:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
print(pipelineFit)

PipelineModel_773ac326d34e


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col, split, explode, when
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
import time

if __name__ == "__main__":
    spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

    # read the tweet data from socket
    tweet_daf = spark \
        .readStream \
        .format("socket") \
        .option("host", "0.0.0.0") \
        .option("port", 5005) \
        .load()

    # type cast the column value
tweet_df = tweet_daf.select(explode(split(col("value"), "TWEET__AN,")).alias("value"))
tweet_df_string = (tweet_df.withColumn('tweet', split(tweet_df['value'], '____SEP,').getItem(0)).withColumn('location', split(tweet_df['value'], '____SEP,').getItem(1)).withColumn('favorites', split(tweet_df['value'], '____SEP,').getItem(2)).withColumn('followers', split(tweet_df['value'], '____SEP,').getItem(3)).withColumn('timestamp', split(tweet_df['value'], '____SEP,').getItem(4)).withColumn('name', split(tweet_df['value'], '____SEP,').getItem(5)))
    
tweet_df_string =tweet_df_string.drop("value")
tweet_df_sentiment = pipelineFit.transform(tweet_df_string).drop("tokens","filtered_words","vector","rawPrediction","probability")
tweet_df_sentiment=tweet_df_sentiment.withColumnRenamed("prediction","sentiment")
tweet_df_fl = pipelineFitBBC.transform(tweet_df_sentiment).drop("tokens","filtered_words","vector","rawPrediction","probability","cf")
tweet_df_fl = tweet_df_fl.withColumn("favorites",col("favorites").cast("int")+1)
tweet_df_fl = tweet_df_fl.withColumn("followers",col("followers").cast("int"))
tweet_df_fl = tweet_df_fl.withColumn("prediction", when(tweet_df_fl.prediction == 0,"tech") \
          .when(tweet_df_fl.prediction == 1,"business").when(tweet_df_fl.prediction == 2,"politics").when(tweet_df_fl.prediction == 3,"sport").when(tweet_df_fl.prediction == 4,"entertainment").otherwise("Null"))
tweet_df_fl = tweet_df_fl.withColumn("sentiment", when(tweet_df_fl.sentiment == 0.0,"Negative") .when(tweet_df_fl.sentiment == 4.0,"Positive").otherwise("Null"))
    
    # write the above data into memory. consider the entire analysis in all iteration (output mode = complete). and let the trigger runs in every 2 secs.
writeTweet = tweet_df_fl.writeStream.format("memory").outputMode("append").queryName("processed").trigger(processingTime ='30 seconds').start()
print("----- streaming is running -------")


23/05/02 21:33:25 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
23/05/02 21:33:26 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
23/05/02 21:33:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/ny/vz9x996d1tsb28njj8vbv9fw0000gn/T/temporary-3998a465-07c1-437d-ac6c-6352ca6b2a82. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/05/02 21:33:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
----- streaming is running -------


In [5]:
# spark.stop()

In [6]:
tweets = spark.sql("select * from processed where tweet!=''")

In [7]:
tweets.count()

0

In [8]:
tweets.head(10)

[]

In [9]:
!pip install plotly



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [10]:
%pylab inline
import plotly.express as px

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [11]:
!pip install dash==0.29.0


  Using cached dash-0.29.0-py3-none-any.whl
  Attempting uninstall: dash
    Found existing installation: dash 2.9.3
    Uninstalling dash-2.9.3:
      Successfully uninstalled dash-2.9.3

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [12]:
!pip install dash-core-components
!pip install --upgrade dash dash-core-components dash-html-components dash-renderer


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
  Using cached dash-2.9.3-py3-none-any.whl (10.2 MB)
  Attempting uninstall: dash
    Found existing installation: dash 0.29.0
    Uninstalling dash-0.29.0:
      Successfully uninstalled dash-0.29.0

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [13]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.graph_objects as go

import plotly.express as px
PORT = 5005
data = spark.sql("select * from processed").groupBy("sentiment").count().toPandas()

data.info()
#data.info()
app = dash.Dash(__name__)
app.layout = html.Div(
    children=[
        html.H1(children="LIVE DASHBOARD",),
        html.H3(children="Top 3 Tweets From Most Influenced People on live",)
        , dcc.Interval('userwithtopfollwers-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='userwithtopfollwers-graph'),
        html.H3(children="Top 3 liked Tweets on live",),
        dcc.Interval('tweetwithtoplikes-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='tweetwithtoplikes-graph'),
        
        html.H3(children="Overall Positive and Negative tweets classification",),
        dcc.Interval('sentiment-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='sentiment-graph'),
        
        html.H3(children="Overall Tweets in each Category",),
        dcc.Interval('category-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='category-graph'),
        
        html.H3(children="Overall Positive and Negative Tweets in each Category",),
        dcc.Interval('sentiment-category-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='sentiment-category-graph'),
        
        html.H3(children="Total likes for Tweets based on Sentiment",),
        dcc.Interval('weight-graph-update', interval = 20000, n_intervals = 0),
        dcc.Graph(id='weight-graph'),
       
        
    ]
)
@app.callback(
        dash.dependencies.Output('sentiment-graph','figure'),
        [dash.dependencies.Input('sentiment-graph-update', 'n_intervals')])
def display_structure(n):
    df = spark.sql("select * from processed where tweet!=''").groupBy("sentiment").count().toPandas()
    fig = px.bar(df, x="sentiment", y="count")
    return fig

@app.callback(
        dash.dependencies.Output('category-graph','figure'),
        [dash.dependencies.Input('category-graph-update', 'n_intervals')])
def display_structure_2(n):
    df = spark.sql("select * from processed where tweet!=''").groupBy("prediction").count().toPandas()
    fig = px.pie(df, values='count', names='prediction',)
    return fig

@app.callback(
        dash.dependencies.Output('sentiment-category-graph','figure'),
        [dash.dependencies.Input('sentiment-category-graph-update', 'n_intervals')])
def display_structure_3(n):
    df = spark.sql("select * from processed where tweet!=''").groupBy("sentiment","prediction").count().toPandas()
    fig = px.bar(df, x="prediction", y="count", color="sentiment",title="Overall Positive and Negative Tweets in each Category")
    return fig

@app.callback(
        dash.dependencies.Output('weight-graph','figure'),
        [dash.dependencies.Input('weight-graph-update', 'n_intervals')])
def display_structure_4(n):
    df = spark.sql("select sentiment,count(favorites) as count from processed where tweet!='' group by sentiment").toPandas()
    #df = df.groupby("sentiment").favorites.sum()
    #fig = px.bar(df, x="favorites", y="followers")
    fig = px.bar(df,x="sentiment",y="count", color="sentiment", title="Total likes for Tweets based on Sentiment")
    return fig

@app.callback(
        dash.dependencies.Output('userwithtopfollwers-graph','figure'),
        [dash.dependencies.Input('userwithtopfollwers-graph-update', 'n_intervals')])
def display_structure_5(n):
    df = tweetPandas=spark.sql("select tweet,name,sentiment,followers from processed where tweet!='' order by followers desc limit 3").toPandas()
    #df = df.groupby("sentiment").favorites.sum()
    #fig = px.bar(df, x="favorites", y="followers")
    fig = go.Figure(data=[go.Table(header=dict(values=list(df.columns),fill_color='paleturquoise',align='left'),cells=dict(values=[df.tweet, df.name, df.sentiment, df.followers],fill_color='lavender',align='left'))])
    return fig

@app.callback(
        dash.dependencies.Output('tweetwithtoplikes-graph','figure'),
        [dash.dependencies.Input('tweetwithtoplikes-graph-update', 'n_intervals')])
def display_structure_6(n):
    df = tweetPandas=spark.sql("select tweet,name,sentiment,favorites from processed where tweet!='' order by favorites desc limit 3").toPandas()
    #df = df.groupby("sentiment").favorites.sum()
    #fig = px.bar(df, x="favorites", y="followers")
    fig = go.Figure(data=[go.Table(header=dict(values=list(df.columns),fill_color='paleturquoise',align='left'),cells=dict(values=[df.tweet, df.name, df.sentiment, df.favorites],fill_color='lavender',align='left'))])
    return fig

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentiment  0 non-null      object
 1   count      0 non-null      int64 
dtypes: int64(1), object(1)
memory usage: 124.0+ bytes


In [ ]:
if __name__ == '__main__':
    app.run_server(
        port=PORT)

Dash is running on http://127.0.0.1:5005/



INFO:dash.dash:Dash is running on http://127.0.0.1:5005/



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5005
INFO:werkzeug:Press CTRL+C to quit
